### Imports

In [1]:
import sys
#sys.path.insert(0, "/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")
#sys.path.remove("/work/pi_wenlongzhao_umass_edu/27/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")

print(sys.path)


['/work/pi_wenlongzhao_umass_edu/27/atifabedeen/pipeline/coir_main', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python38.zip', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/lib-dynload', '', '/home/aabedeen_umass_edu/.local/lib/python3.8/site-packages', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages']


In [2]:
if 'coir' in sys.modules:
    del sys.modules['coir']
    print("success")

In [3]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
import pandas as pd

### getting the corpus & queries

In [8]:
tasks = get_tasks(tasks=["CodeSearchNet-python"])

0it [00:00, ?it/s]

In [13]:
len(tasks['CodeSearchNet-python'][0])

280310

In [5]:
#expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_deepseek_exp.csv'
expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_explanations_query_code.csv'
expl_df = pd.read_csv(expl_path)
expl_df.rename(columns={"query_id": "query-id", "corpus_id": "corpus-id"}, inplace=True)
expl_df.head()

,Unnamed: 0,query-id,doc,corpus-id,code,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4,explanation_deepseek_1_cleaned,explanation_deepseek_2_cleaned,explanation_deepseek_3_cleaned,explanation_deepseek_4_cleaned
0,0,q1,python code to write bool value 1,d1,"def writeBoolean(self, n):\n """"""\n ...",\nThe provided code is a Python method named `...,"\n1. `def writeBoolean(self, n):` - This line ...",\nThis code snippet is a function that writes ...,\nThe code snippet provided is a method named ...,The doc string and code both write the boolean...,"Alright, I need to break down this Python code...",The code snippet writes a boolean value to a s...,The code snippet defines a method called write...
1,1,q2,"""python how to manipulate clipboard""",d2,"def paste(xsel=False):\n """"""Returns system ...",\nThe provided Python code snippet defines a f...,\n1. `def paste(xsel=False):` - This line defi...,\nThis code snippet is a function in Python th...,\nThe code snippet provided is a Python functi...,The paste function pastes data from the clipbo...,The code snippet begins by defining a function...,The code snippet is a function called paste th...,The code snippet defines a function called pas...
2,2,q3,python colored output to html,d3,"def _format_json(data, theme):\n """"""Pretty ...","\nThis Python function, `_format_json`, takes ...","\n1. `def _format_json(data, theme):` - This l...","\nThis code snippet, named `_format_json`, is ...",\n```python\nimport json\nimport sys\nimport p...,The docstring explains that the function _form...,"```pythondef _format_json(data, theme): """"""...",This code snippet is a function called _format...,To explain the code snippet in a way that can ...
3,3,q4,"python ""create directory"" using ""relative path""",d4,"def create_path(path):\n """"""Creates a absol...","\nThe provided Python function, `create_path(p...",\n1. The code snippet defines a function named...,\nThis code snippet helps you create a new dir...,\nThe provided code snippet is a Python functi...,The doc and code create an absolute path from ...,"Okay, I need to explain this Python code snipp...",This code creates a file system where any path...,The code snippet defines a function called cre...
4,4,q5,python column of an array,d5,"def _vector_or_scalar(x, type='row'):\n """"""...",\nThe provided Python code defines a function ...,\nThis code snippet defines a function called ...,\nThis code snippet is a function that convert...,\nThe code snippet provided is a function defi...,The code checks if an object `x` is an array o...,"Alright, so I've got this Python code snippet ...",The code snippet is a Python function called _...,"Okay, I'm going to try to figure out how to ex..."


In [7]:
# for col in expl_df.columns:
#     nan_rows = expl_df[expl_df[col].isna()]
#     if not nan_rows.empty:
#         print(f"Column: {col} - NaN Count: {nan_rows.shape[0]}")
#         #print(nan_rows)  # Print the rows with NaN values for this column
#         print("-" * 50)  # Separator for readability

In [8]:
# expl_df.columns

### adding explanation to df

#### multiple explanations 

In [8]:
corpus, queries, qrels = tasks['cosqa']
new_corpus = {}
new_qrels = {}

for _, row in expl_df.iterrows():
    base_corpus_id = row['corpus-id']  # Existing base document ID (e.g., d1, d2, etc.)
    query_id = row['query-id']         # Query ID (e.g., q1, q2, etc.)
    
    # Iterate over multiple explanations and create new corpus entries
    for i in range(1, 4):  # Assuming 4 explanations per document
        explanation_col = f'explanation_granite_{i}'
        if explanation_col in row and row[explanation_col] and row[explanation_col].strip():
            new_corpus_id = f"{base_corpus_id}_{i}"  # e.g., d1_1, d1_2, ...

            # Store new document in corpus
            new_corpus[new_corpus_id] = {
                "text": row[explanation_col],
                "title": ""
            }

            # Ensure the query is mapped to the new corpus ID
            if query_id not in new_qrels:
                new_qrels[query_id] = {}  # Initialize query entry if missing
            
            new_qrels[query_id][new_corpus_id] = 1  # Assign relevance score

corpus.update(new_corpus)
qrels.update(new_qrels)
tasks['cosqa'] = (corpus, queries, qrels)

print(f"Total docs in expanded corpus: {len(corpus)}")
print(f"Total qrels entries: {sum(len(v) for v in qrels.values())}")

# Print first few new corpus and qrel entries
print(next(iter(new_corpus.items())))  # Print first new corpus entry
print(next(iter(new_qrels.items())))  # Print first new qrel entry


Total docs in expanded corpus: 82416
Total qrels entries: 61812
('d1_1', {'text': "\nThe provided code is a Python method named `writeBoolean` that writes a Boolean value to a stream. It takes one argument `n`, which is a Boolean value. If `n` is `False`, it sets the value of `t` to `TYPE_BOOL_FALSE`. Otherwise, it sets `t` to `TYPE_BOOL_TRUE`. Finally, it writes the value of `t` to the stream using the `write` method of the `self.stream` object. The doc string provides a brief description of the method's purpose and functionality.", 'title': ''})
('q1', {'d1_1': 1, 'd1_2': 1, 'd1_3': 1})


In [ ]:
# corpus, queries, qrels = tasks['cosqa']
# corpus_ids = sorted(corpus, key=lambda k: len(corpus[k].get("title", "") + corpus[k].get("text", "")), reverse=True)
# corpus = [corpus[cid] for cid in corpus_ids]

In [ ]:
# all_texts = ["passage: "+ doc['text'] for doc in corpus]
# len(all_texts)

TypeError: string indices must be integers

#### single explanation

In [6]:
# # Unpack the tuple
corpus, queries, qrels = tasks['cosqa']

for _, row in expl_df.iterrows():
    corpus_id = row['corpus-id']
    explanation = row['explanation_granite_1']

    if corpus_id in corpus and explanation and explanation.strip():
        corpus[corpus_id]['text'] = explanation  

corpus = {doc_id: doc for doc_id, doc in corpus.items() if 'text' in doc and doc['text'].strip()}
print(corpus['d1'])
tasks['cosqa'] = (corpus, queries, qrels)

print(f"Total docs in corpus after replacement: {len(tasks['cosqa'][0])}")
missing_text = sum(1 for doc in tasks['cosqa'][0].values() if not doc.get('text'))
print(f"Number of docs still missing 'text': {missing_text}")


{'text': "\nThe provided code is a Python method named `writeBoolean` that writes a Boolean value to a stream. It takes one argument `n`, which is a Boolean value. If `n` is `False`, it sets the value of `t` to `TYPE_BOOL_FALSE`. Otherwise, it sets `t` to `TYPE_BOOL_TRUE`. Finally, it writes the value of `t` to the stream using the `write` method of the `self.stream` object. The doc string provides a brief description of the method's purpose and functionality.", 'title': ''}
Total docs in corpus after replacement: 20604
Number of docs still missing 'text': 0


In [7]:
print(corpus['d20604'])

{'text': "\nThe provided code snippet is a Python function called `_unzip_handle` that takes a file handle as input and transparently unzips it. If the input is a string, it assumes it's a filename and opens it using `_gzip_open_filename`. If it's a handle, it opens it using `_gzip_open_handle`. The function then returns the unzipped handle. This function is useful for unzipping files in a transparent way, without the need to manually handle the unzipping process.", 'title': ''}


### Running the pipeline

In [11]:
model_name = "intfloat/e5-base-v2"
model = YourCustomDEModel(model_name=model_name)

evaluation = COIR(tasks=tasks,batch_size=128)

results = evaluation.run(model, output_folder=f"results/{model_name}_GRANITE_EXPL_1")
print(results)

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Encoding batches:   0%|          | 0/4 [00:00<?, ?batch/s]

Encoding batches:   0%|          | 0/161 [00:00<?, ?batch/s]

{'cosqa': {'NDCG': {'NDCG@1': 0.236, 'NDCG@3': 0.30438, 'NDCG@5': 0.34216, 'NDCG@10': 0.39245, 'NDCG@100': 0.46143, 'NDCG@1000': 0.47045}, 'MAP': {'MAP@1': 0.236, 'MAP@3': 0.28667, 'MAP@5': 0.30757, 'MAP@10': 0.32786, 'MAP@100': 0.34246, 'MAP@1000': 0.34282}, 'Recall': {'Recall@1': 0.236, 'Recall@3': 0.356, 'Recall@5': 0.448, 'Recall@10': 0.606, 'Recall@100': 0.922, 'Recall@1000': 0.992}, 'Precision': {'P@1': 0.236, 'P@3': 0.11867, 'P@5': 0.0896, 'P@10': 0.0606, 'P@100': 0.00922, 'P@1000': 0.00099}}}


In [9]:
models_list = ["BAAI/bge-base-en", "intfloat/e5-base-v2", "sentence-transformers/all-MiniLM-L6-v2", "microsoft/unixcoder-base", "infly/inf-retriever-v1-1.5b", "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "google-bert/bert-base-uncased"]  # Add your models here

for model_name in models_list:
    model = YourCustomDEModel(model_name=model_name)
    
    evaluation = COIR(tasks=tasks, batch_size=128)
    
    results = evaluation.run(model, output_folder=f"results/{model_name}_explanation_granite_all")
    
    print(f"Results for {model_name}: {results}")


Encoding batches:   0%|          | 0/4 [00:00<?, ?batch/s]

Encoding batches:   0%|          | 0/391 [00:00<?, ?batch/s]

Encoding batches:   0%|          | 0/254 [00:00<?, ?batch/s]

Results for BAAI/bge-base-en: {'cosqa': {'NDCG': {'NDCG@1': 0.036, 'NDCG@3': 0.04745, 'NDCG@5': 0.06309, 'NDCG@10': 0.08603, 'NDCG@100': 0.19709, 'NDCG@1000': 0.26269}, 'MAP': {'MAP@1': 0.012, 'MAP@3': 0.03233, 'MAP@5': 0.04503, 'MAP@10': 0.05464, 'MAP@100': 0.07881, 'MAP@1000': 0.08239}, 'Recall': {'Recall@1': 0.012, 'Recall@3': 0.05, 'Recall@5': 0.07667, 'Recall@10': 0.128, 'Recall@100': 0.52533, 'Recall@1000': 0.89667}, 'Precision': {'P@1': 0.036, 'P@3': 0.05, 'P@5': 0.046, 'P@10': 0.0384, 'P@100': 0.01576, 'P@1000': 0.00269}}}


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Encoding batches:   0%|          | 0/4 [00:00<?, ?batch/s]

Encoding batches:   0%|          | 0/391 [00:00<?, ?batch/s]

Encoding batches:   0%|          | 0/254 [00:00<?, ?batch/s]

Results for google-bert/bert-base-uncased: {'cosqa': {'NDCG': {'NDCG@1': 0.006, 'NDCG@3': 0.00435, 'NDCG@5': 0.00552, 'NDCG@10': 0.00646, 'NDCG@100': 0.01305, 'NDCG@1000': 0.02399}, 'MAP': {'MAP@1': 0.002, 'MAP@3': 0.00278, 'MAP@5': 0.00324, 'MAP@10': 0.00353, 'MAP@100': 0.00439, 'MAP@1000': 0.00463}, 'Recall': {'Recall@1': 0.002, 'Recall@3': 0.004, 'Recall@5': 0.006, 'Recall@10': 0.008, 'Recall@100': 0.032, 'Recall@1000': 0.09867}, 'Precision': {'P@1': 0.006, 'P@3': 0.004, 'P@5': 0.0036, 'P@10': 0.0024, 'P@100': 0.00096, 'P@1000': 0.0003}}}
